# ETL

In [1]:
import pandas as pd
# Se configura para que se muestre la totalidad de las columnas del dataframe.
pd.options.display.max_columns = None

## Hechos  
Se extraen los datos de la hoja "HECHOS" del archivo "homicidios.xlxs".

In [2]:
hechos = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\PI_02.zip\\PI-Data Analytics\\Data\\homicidios.xlsx'
df_hechos = pd.read_excel(hechos, engine='openpyxl', sheet_name='HECHOS')

In [3]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [4]:
df_hechos.sample()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
69,2016-0093,1,2016-07-17,2016,7,17,03:30:00,3,AV JUAN B ALBERDI Y EMILIO MITRE,AVENIDA,"ALBERDI, JUAN BAUTISTA AV.",NaN,"MITRE, EMILIO","ALBERDI, JUAN BAUTISTA AV. y MITRE, EMILIO",6,Point (101804.61714036 100693.45218194),-58.44362126,-34.62301748,AUTO-AUTO,AUTO,AUTO


Se normalizan los nombres de las columnas de la hoja de "HECHOS" para brindar una mayor consistencia.

In [5]:
df_hechos = df_hechos.rename(columns={
                                        'ID': 'ID Siniestro',
                                        'N_VICTIMAS': 'Número de víctimas',
                                        'FECHA': 'Fecha',
                                        'AAAA': 'Año',
                                        'MM': 'Mes',
                                        'DD': 'Día',
                                        'HORA': 'Hora',
                                        'HH':'Hora entera',
                                        'LUGAR_DEL_HECHO':'Lugar del hecho',
                                        'TIPO_DE_CALLE':'Tipo de calle',
                                        'COMUNA':'Comuna',
                                        'XY(CABA)':'Geolocación',
                                        'pos x':'Coordenada X',
                                        'pos y':'Coordenada Y',
                                        'PARTICIPANTES':'Participantes',
                                        'VICTIMA':'Víctima',
                                        'ACUSADO':'Acusado',
                                        })

In [6]:
df_hechos.sample()

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
678,2021-0080,1,2021-10-28,2021,10,28,16:16:00,16,ARENALES Y 9 DE JULIO AV.,AVENIDA,ARENALES,NaN,9 DE JULIO AV.,ARENALES y 9 DE JULIO AV.,1,Point (107387.05825032 103876.26437289),-58.38277479,-34.59430160,MOTO-AUTO,MOTO,AUTO


### Valores nulos.

In [7]:
nulos_hechos = df_hechos.isnull().sum()
nulos_hechos


ID Siniestro               0
Número de víctimas         0
Fecha                      0
Año                        0
Mes                        0
Día                        0
Hora                       0
Hora entera                0
Lugar del hecho            0
Tipo de calle              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
Comuna                     0
XY (CABA)                  0
Coordenada X               0
Coordenada Y               0
Participantes              0
Víctima                    0
Acusado                    0
dtype: int64

### Filas duplicadas.

In [8]:
hechos_duplicados = df_hechos.duplicated()
hechos_duplicados.sum()

0

El dataframe no presenta filas duplicadas.

### Manejo de columnas.

**Columna "ID Siniestro".**

In [9]:
tipos_id = df_hechos['ID Siniestro'].apply(type).value_counts()
tipos_id

ID Siniestro
<class 'str'>    696
Name: count, dtype: int64

Se puede observar que la columna "ID Siniestro" esta confomrada únicamente por valores de tipo string.

In [10]:
id_unicos = df_hechos['ID Siniestro'].nunique()
id_unicos

696

Y, a su vez, los valores corresponden a números de identificación unicos para cada fila.

**Columna "Hora".**

In [11]:
tipos_hora = df_hechos['Hora'].apply(type).value_counts()
tipos_hora

Hora
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

Se crea una función que permite convertir los valores que no están en formato "datetime.time" a este formato. Y de no se posible se lo convierte a formato "NaT".

In [12]:
def convert_to_time(x):
    try:
        return pd.to_datetime(x, format='%H:%M:%S', errors='raise').time()
    except (ValueError, TypeError):
        return pd.NaT
df_hechos['Hora'] = df_hechos['Hora'].apply(convert_to_time)

In [13]:
tipos_hora = df_hechos['Hora'].apply(type).value_counts()
tipos_hora

Hora
<class 'datetime.time'>                          695
<class 'pandas._libs.tslibs.nattype.NaTType'>      1
Name: count, dtype: int64

In [14]:
df_hechos[df_hechos['Hora'].isna()]

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,NaT,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Se imputa el valor faltante de la columna por el valor modal de la misma.

In [15]:
moda = df_hechos['Hora'].mode()[0]
df_hechos['Hora'] = df_hechos['Hora'].apply(lambda x: moda if pd.isna(x) else x)

In [16]:
tipos_hora = df_hechos['Hora'].apply(type).value_counts()
tipos_hora

Hora
<class 'datetime.time'>    696
Name: count, dtype: int64

**Columna "Hora entera".**

In [17]:
tipos_hora_e = df_hechos['Hora entera'].apply(type).value_counts()
tipos_hora_e

Hora entera
<class 'int'>    695
<class 'str'>      1
Name: count, dtype: int64

In [18]:
valores_unicos = df_hechos['Hora entera'].unique()
print(f'Los valores unicos de la columna son: {(valores_unicos)}')

Los valores unicos de la columna son: [4 1 7 0 5 18 19 15 11 22 16 9 23 6 10 17 12 8 20 21 14 3 2 13 'SD']


In [19]:
df_hechos[df_hechos['Hora entera'] == 'SD']

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,09:00:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


El valor string de la columna  "Hora entera" corresponde a la misma fila a la cual se le reemplazó el valor faltante en la columna "Hora", por lo que se procede a imputar el valor "SD" por "9".

In [20]:
df_hechos.loc[df_hechos['Hora entera'] == 'SD', 'Hora entera'] = 9

In [21]:
df_hechos[df_hechos['ID Siniestro'] == '2019-0103']

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,09:00:00,9,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


**Nueva columna "Momento del día"**  
Se crea una nueva columna a partir de la columna "Hora entera" para identificar el momento del día en el que se dió el siniestro. Los momentos del dia serán considerados como "Madrugada"(para el período entre las 0 y las 6 hs), "Mañana"(para el período entre las 6 y las 12 hs), "Tarde"(para el período entre las 12 y las 18 hs) y "Noche"(para el período entre las 18 y las 24 hs).  
También se realiza una reindexación para que esté junto a las columnas temporales.

In [22]:
madrugada = range(0, 6)
mañana = range(6, 12)
tarde = range(12, 18)
noche = range(18, 24)

df_hechos['Momento del día'] = df_hechos['Hora entera'].apply(
    lambda x: 'Madrugada' if x in madrugada else
            'Mañana' if x in mañana else
            'Tarde' if x in tarde else
            'Noche'
            )

nuevo_orden = ['ID Siniestro', 'Número de víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora', 'Hora entera', 'Momento del día', 'Lugar del hecho',
                'Tipo de calle','Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'Comuna', 'XY (CABA)',	'Coordenada X',	
                'Coordenada Y', 'Participantes', 'Víctima', 'Acusado']

df_hechos = df_hechos.reindex(columns=nuevo_orden)

df_hechos.head(1)

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,Madrugada,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


**Colunma "Calle".**

In [23]:
tipos_calle = df_hechos['Calle'].apply(type).value_counts()
tipos_calle

Calle
<class 'str'>      695
<class 'float'>      1
Name: count, dtype: int64

In [24]:
filas_float = df_hechos['Calle'].apply(lambda x: isinstance(x, float))
df_hechos[filas_float]

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,Noche,SD,CALLE,NaN,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


Al tratarse de un valor de tipo "NaN" se procede a imputarlo por "SD".

In [25]:
df_hechos['Calle'] = df_hechos['Calle'].fillna('SD')

In [26]:
df_hechos[df_hechos['ID Siniestro'] == '2016-0151']

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,Noche,SD,CALLE,SD,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


**Columnas "Altura" y "Cruce".**

In [27]:
tipos_cruce = df_hechos['Cruce'].apply(type).value_counts()
tipos_altura = df_hechos['Altura'].apply(type).value_counts()
print(tipos_cruce)
print(tipos_altura)

Cruce
<class 'str'>      525
<class 'float'>    171
Name: count, dtype: int64
Altura
<class 'float'>    696
Name: count, dtype: int64


In [28]:
df_hechos[['Altura', 'Cruce']].head(10)

,Altura,Cruce
0,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,DE LOS CORRALES AV.
2,2034.0,NaN
3,NaN,"VILLEGAS, CONRADO, GRAL."
4,NaN,"SAENZ PE?A, LUIS, PRES."
5,NaN,ESCALADA AV.
6,NaN,"GONZALEZ, JOAQUIN V."
7,NaN,DE LOS CORRALES AV.
8,NaN,"IRIGOYEN, BERNARDO DE"
9,1366.0,NaN


Se puede observar que en los campos que estan completos por valores no nulos en la columna "Altura" están vacíos en la columna "Calle" y viceversa. Como la columna que posee la menor cantidad de nulos es la columna "Cruce", se procede a eliminar la columna "Altura" e imputar los valores no nulos por el valor "Si" y los valores nulos por "No" para señalar los siniestros que se dieron en cruces y los que no.

In [29]:
df_hechos.drop(['Altura'],axis=1, inplace=True)
df_hechos['Cruce'] = df_hechos['Cruce'].fillna('No')
df_hechos.loc[df_hechos['Cruce'] != 'No', 'Cruce'] = 'Sí'

In [30]:
df_hechos[['Cruce']].head(10)

,Cruce
0,Sí
1,Sí
2,No
3,Sí
4,Sí
5,Sí
6,Sí
7,Sí
8,Sí
9,No


**Columna "Dirección Normalizada".**

In [31]:
tipos_direc = df_hechos['Dirección Normalizada'].apply(type).value_counts()
tipos_direc

Dirección Normalizada
<class 'str'>      688
<class 'float'>      8
Name: count, dtype: int64

In [32]:
filas_float = df_hechos['Dirección Normalizada'].apply(lambda x: isinstance(x, float))
df_hechos[filas_float]

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,Noche,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",No,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,Madrugada,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,No,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,Noche,SD,CALLE,SD,No,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,Mañana,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,No,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,Madrugada,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,No,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,Noche,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",No,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,Mañana,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",No,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,Mañana,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,No,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


Se procede a imputar los valores nulos por "SD".

In [33]:
df_hechos['Dirección Normalizada'] = df_hechos['Dirección Normalizada'].fillna('SD')

In [34]:
tipos_direc = df_hechos['Dirección Normalizada'].apply(type).value_counts()
tipos_direc

Dirección Normalizada
<class 'str'>    696
Name: count, dtype: int64

**Columnas de geolocación.**

In [35]:
filas_caba = df_hechos[df_hechos['XY (CABA)'] == 'Point (. .)']
filas_caba.shape


(14, 21)

La cantidad de filas en las que está presente el valor "Point (. .)" en la columna "XY (CABA)" es 14.

In [36]:
filas_caba[['XY (CABA)', 'Coordenada X', 'Coordenada Y']].head(14)

,XY (CABA),Coordenada X,Coordenada Y
35,Point (. .),-58.37714647568196,-34.63657525428238
38,Point (. .),.,.
71,Point (. .),-58.47433193007387,-34.66684950051973
106,Point (. .),.,.
119,Point (. .),.,.
139,Point (. .),.,.
176,Point (. .),.,.
180,Point (. .),.,.
181,Point (. .),.,.
256,Point (. .),.,.


A su vez las columnas "Coordenada X" y "Coordenada Y" también presentan algunas filas en las que está presente el valor "." en lugar de una coordenada.

In [37]:
filas_xy = df_hechos[(df_hechos['Coordenada X'] == '.') & (df_hechos['Coordenada Y'] == '.')]
filas_xy.shape

(12, 21)

La cantidad de filas con dicho valor corresponden a las del dataframe mostrado anteriormente.   
Como no hay manera de completar esos campos con el valor correspondiente, se opta por imputar los valores por 0 de las columnas "XY (CABA)" ,"Coordenada X" y "Coordenada Y".

In [38]:
df_hechos['Coordenada X'] = df_hechos['Coordenada X'].replace('.', 0)
df_hechos['Coordenada Y'] = df_hechos['Coordenada Y'].replace('.', 0)
df_hechos['XY (CABA)'] = df_hechos['XY (CABA)'].replace('Point (. .)', 0)

In [39]:
filas_cero = df_hechos[df_hechos['XY (CABA)'] == 0]
print(filas_cero.shape)
filas_cero[['XY (CABA)', 'Coordenada X', 'Coordenada Y']].head(14)

(14, 21)


,XY (CABA),Coordenada X,Coordenada Y
35,0,-58.37714647568196,-34.63657525428238
38,0,0,0
71,0,-58.47433193007387,-34.66684950051973
106,0,0,0
119,0,0,0
139,0,0,0
176,0,0,0
180,0,0,0
181,0,0,0
256,0,0,0


**Columna "Víctima"**

In [40]:
vict_unicos = df_hechos['Víctima'].unique()
print(vict_unicos)

['MOTO' 'AUTO' 'PEATON' 'SD' 'CARGAS' 'BICICLETA' 'PASAJEROS' 'MOVIL'
 'OBJETO FIJO' 'PEATON_MOTO']


In [41]:
filas_vict = df_hechos[df_hechos['Víctima'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]
filas_vict

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,Madrugada,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",Sí,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,Mañana,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,Sí,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


En el diccionario de datos de Siniestros Viales no se encuentran los valores "OBJETO FIJO" ni "PEATON_MOTO", por lo que se procede a imputar dichos datos por el valor "OTRO", que sí figura en el diccionario.

In [42]:
df_hechos['Víctima'] = df_hechos['Víctima'].replace({'OBJETO FIJO': 'OTRO', 'PEATON_MOTO': 'OTRO'})

A su vez se modifican los valores "AUTO-OBJETO FIJO" y "PEATON_MOTO-MOTO" de la columna "Participantes" para adecuarlos al cambio realizado en la columna "Víctima"

In [43]:
df_hechos['Participantes'] = df_hechos['Participantes'].replace({'AUTO-OBJETO FIJO': 'OTRO-AUTO', 'PEATON_MOTO-MOTO': 'OTRO-MOTO'})

In [44]:
filas_vict = df_hechos[df_hechos['ID Siniestro'].isin(['2017-0108', '2020-0063'])]
filas_vict

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,Madrugada,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",Sí,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,OTRO-AUTO,OTRO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,Mañana,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,Sí,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,OTRO-MOTO,OTRO,MOTO


**Columna "Acusado"**

In [45]:
acus_unico = df_hechos['Acusado'].unique()
print(acus_unico)

['AUTO' 'PASAJEROS' 'SD' 'OBJETO FIJO' 'CARGAS' 'MOTO' 'MULTIPLE' 'OTRO'
 'BICICLETA' 'TREN']


Todos los datos únicos que se visualizan en la columna "Acusado" corresponden a los que se describen en el diccionario de siniestros viales.

**Columna "Participantes"**

In [46]:
part_unicos = df_hechos['Participantes'].unique()
print(part_unicos)

['MOTO-AUTO' 'AUTO-PASAJEROS' 'MOTO-SD' 'MOTO-PASAJEROS'
 'MOTO-OBJETO FIJO' 'MOTO-CARGAS' 'PEATON-AUTO' 'PEATON-MOTO'
 'PEATON-PASAJEROS' 'AUTO-AUTO' 'OTRO-AUTO' 'PEATON-CARGAS' 'MOTO-MOTO'
 'MULTIPLE' 'SD-SD' 'AUTO-CARGAS' 'AUTO-SD' 'MOTO-MOVIL' 'CARGAS-CARGAS'
 'PEATON-SD' 'BICICLETA-CARGAS' 'SD-CARGAS' 'BICICLETA-PASAJEROS'
 'PASAJEROS-PASAJEROS' 'MOVIL-PASAJEROS' 'CARGAS-OBJETO FIJO'
 'PEATON-BICICLETA' 'SD-AUTO' 'CARGAS-AUTO' 'BICICLETA-AUTO'
 'MOTO-BICICLETA' 'SD-MOTO' 'MOVIL-CARGAS' 'PASAJEROS-AUTO' 'AUTO-MOVIL'
 'CARGAS-PASAJEROS' 'BICICLETA-TREN' 'BICICLETA-OTRO' 'MOTO-OTRO'
 'OTRO-MOTO' 'PASAJEROS-SD']


Todos los datos en la columna "Participantes" corresponden a los estipulados en el diccionario.

## Víctimas  
Se extraen los datos de la hoja "VICTIMAS" del archivo "homicidios.xlxs".

In [47]:
victimas = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\PI_02.zip\\PI-Data Analytics\\Data\\homicidios.xlsx'
df_victimas = pd.read_excel(victimas, engine='openpyxl', sheet_name='VICTIMAS')

In [48]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Se normalizan los nombres de las columnas de la hoja de "VICTIMAS" para brindar una mayor consistencia. 

In [49]:
df_victimas = df_victimas.rename(columns={
                                        'ID_hecho': 'ID Siniestro',
                                        'FECHA': 'Fecha',
                                        'AAAA': 'Año',
                                        'MM': 'Mes',
                                        'DD': 'Día',
                                        'ROL': 'Rol víctima',
                                        'VICTIMA':'Víctima',
                                        'SEXO':'Sexo',
                                        'EDAD':'Edad',
                                        'FECHA_FALLECIMIENTO':'Fecha Fallecimiento'
                                        })

In [50]:
df_victimas.sample()

,ID Siniestro,Fecha,Año,Mes,Día,Rol víctima,Víctima,Sexo,Edad,Fecha Fallecimiento
103,2016-0131,2016-10-09,2016,10,9,PEATON,PEATON,MASCULINO,35,SD


### Valores nulos

In [51]:
nulos_victimas = df_victimas.isnull().sum()
nulos_victimas

ID Siniestro           0
Fecha                  0
Año                    0
Mes                    0
Día                    0
Rol víctima            0
Víctima                0
Sexo                   0
Edad                   0
Fecha Fallecimiento    0
dtype: int64

El dataframe no presenta columnas con valores nulos.

### Filas duplicadas.

In [52]:
victimas_duplicados = df_victimas.duplicated()
victimas_duplicados.sum()

0

El dataframe no presenta filas duplicadas.

### Manejo de columnas.

**Eliminación de columans innecesarias**

Se opta eliminar las columnas que se comparten con "df_hechos" ya que se hará un merge con "df_hechos" y se quedarán las columnas que se necesiten. Y también se decide eliminar la columna "Fecha Fallecimiento" ya que se vuelve redundante al presentar los mismos datos que las columns "Fecha" y "Hora".

In [53]:
df_victimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Víctima', 'Fecha Fallecimiento'],axis=1, inplace=True)
df_victimas.sample()

,ID Siniestro,Rol víctima,Sexo,Edad
593,2020-0054,CICLISTA,MASCULINO,59


**Columna "ID Siniestro"**

In [54]:
tipos_ID = df_victimas['ID Siniestro'].apply(type).value_counts()
tipos_ID

ID Siniestro
<class 'str'>    717
Name: count, dtype: int64

In [55]:
id_dup= df_victimas.duplicated(subset=['ID Siniestro'], keep=False).sum()
id_dup

41

El dataframe presenta 41 números ID duplicados.

In [56]:
filas_dup_id = df_victimas[df_victimas.duplicated(subset=['ID Siniestro'], keep=False)]
filas_dup_id.head(6)

,ID Siniestro,Rol víctima,Sexo,Edad
29,2016-0041,CONDUCTOR,MASCULINO,54
30,2016-0041,PASAJERO_ACOMPAÑANTE,MASCULINO,SD
98,2016-0126,CONDUCTOR,MASCULINO,37
99,2016-0126,PASAJERO_ACOMPAÑANTE,MASCULINO,60
163,2017-0026,PASAJERO_ACOMPAÑANTE,FEMENINO,23
164,2017-0026,CONDUCTOR,MASCULINO,19


Se puede apreciar que el motivo por el que hay números ID duplicados se debe a que son siniestros en los que se implicó como víctima fatal a más de una persona.

**Columna "Sexo"**

In [57]:
tipos_sexo = df_victimas['Sexo'].apply(type).value_counts()
tipos_sexo

Sexo
<class 'str'>    717
Name: count, dtype: int64

In [58]:
sexo_unicos = df_victimas['Sexo'].unique()
print(sexo_unicos)

['MASCULINO' 'FEMENINO' 'SD']


In [59]:
sexo_sd = df_victimas[df_victimas['Sexo'].isin(['SD'])]
print(sexo_sd.shape)
sexo_sd

(6, 4)


,ID Siniestro,Rol víctima,Sexo,Edad
36,2016-0049,SD,SD,SD
39,2016-0052,SD,SD,SD
108,2016-0136,CONDUCTOR,SD,SD
121,2016-0151,PEATON,SD,SD
138,2016-0171,CONDUCTOR,SD,SD
141,2016-0174,SD,SD,SD


La columna "Sexo" presenta seis valores de tipo "SD", por lo que se decide imputar dichos valores por el valor modal.

In [60]:
moda = df_victimas['Sexo'].mode()[0]
df_victimas['Sexo'] = df_victimas['Sexo'].replace('SD', moda)
print(f'El valor mas frecuente la columna "Sexo" es: {moda}')

El valor mas frecuente la columna "Sexo" es: MASCULINO


Se verifica que el cambio se haya efectuado.

In [61]:
sexo_sd = df_victimas[df_victimas['Sexo'].isin(['SD'])]
print(sexo_sd.shape)
sexo_sd

(0, 4)


,ID Siniestro,Rol víctima,Sexo,Edad


**Columna "Edad"**

In [62]:
tipos_edad = df_victimas['Edad'].apply(type).value_counts()
tipos_edad

Edad
<class 'int'>    664
<class 'str'>     53
Name: count, dtype: int64

Se aprecia que la columna presenta valores de tipo "string" cuando solo deberia presentar valores numéricos.

In [63]:
valores_str = df_victimas['Edad'][df_victimas['Edad'].apply(lambda x: isinstance(x, str))]
valor_unico_str = valores_str.unique()
print(valor_unico_str)

['SD']


Los 53 valores de tipo "string" que se encuentran en la columna corresponden al valor "SD". Se decide imputar los datos faltantes con por la media de edad teniendo en cuenta el rol de la víctima.

In [64]:
# Se convierten los valores no numéricos a valores nulos.
df_victimas['Edad'] = pd.to_numeric(df_victimas['Edad'], errors='coerce')

# Se calcula la media de la edad por grupo
edad_promedio_por_grupo = df_victimas.groupby('Rol víctima')['Edad'].mean()

# Se define una función para reemplazar los valores nulos de la columna "Edad" por la media de la edad por grupo.
def reemplazar_sd(row):
    if pd.isna(row['Edad']):
        return edad_promedio_por_grupo[row['Rol víctima']]
    else:
        return row['Edad']

# Se aplica la función a los elementos de la columna "Edad".
df_victimas['Edad'] = df_victimas.apply(reemplazar_sd, axis=1)

#Se convierten los valores de la columna "Edad" a tipo entero.
df_victimas['Edad'] = df_victimas['Edad'].astype(int)

Se verifica que se hayan efectuado los cambios.

In [65]:
tipos_edad = df_victimas['Edad'].apply(type).value_counts()
tipos_edad

Edad
<class 'int'>    717
Name: count, dtype: int64

In [66]:
df_victimas.head()

,ID Siniestro,Rol víctima,Sexo,Edad
0,2016-0001,CONDUCTOR,MASCULINO,19
1,2016-0002,CONDUCTOR,MASCULINO,70
2,2016-0003,CONDUCTOR,MASCULINO,30
3,2016-0004,CONDUCTOR,MASCULINO,18
4,2016-0005,CONDUCTOR,MASCULINO,29


### Merge de dataframes.

In [67]:
df_homicidios = df_hechos.merge( df_victimas, on='ID Siniestro', how='left')
df_homicidios.head()

,ID Siniestro,Número de víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Momento del día,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Coordenada X,Coordenada Y,Participantes,Víctima,Acusado,Rol víctima,Sexo,Edad
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,Madrugada,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,Sí,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,CONDUCTOR,MASCULINO,19
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,Madrugada,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",Sí,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,CONDUCTOR,MASCULINO,70
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,Mañana,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,No,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,CONDUCTOR,MASCULINO,30
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,Madrugada,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,Sí,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,CONDUCTOR,MASCULINO,18
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,Madrugada,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,Sí,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,CONDUCTOR,MASCULINO,29


### Carga de datos.

In [68]:
df_homicidios.to_csv('homicidios.csv', index=False)